Set the working directory to the one above to import our bw2data version

In [1]:
import os, pathlib
os.chdir(pathlib.Path(os.getcwd()).parent.absolute())

In [2]:
import bw2data as bd
import bw2calc as bc

Set-up the project for a clean start

In [3]:
from bw2data.backends.schema import ActivityDataset, ExchangeDataset, ProductDataset

# set project en clear database
bd.projects.set_current("MF_TEST")
bd.backends.sqlite3_lci_db.db.drop_tables([ActivityDataset, ProductDataset, ExchangeDataset])
bd.backends.sqlite3_lci_db.db.create_tables([ActivityDataset, ProductDataset, ExchangeDataset])

Realy simple ecosystem with a single biosphere flow and a single method for said bioshpere flow

In [4]:
# setup a mockup project
eco_db = bd.Database("eco")
eco_db.write({("eco", "carbon_dioxide"): {"name": "carbon_dioxide", "type": "emission"}})

mthd_0 = bd.Method(("CO2eq",))
mthd_0.register()
mthd_0.write([(("eco", "carbon_dioxide"), 1)])

100%|██████████| 1/1 [00:00<?, ?it/s]

Vacuuming database 


Import a simple iron production model with a multifunctional mining process. You can add products using the standard `SQLiteBackend.write` functionality if you use the format described below:

In [5]:
model_db = bd.Database("model")
model_db.write(
    {
        ("model", "electricity_production"): {
            "name": "electricity_production", 
            "type": "process", 
            "exchanges": [
                {"input": ("eco", "carbon_dioxide"),"amount": 1.0, "type": "biosphere"},
                {"input": ("model", "coal"),"amount": 1.0, "type": "technosphere"},
                {"input": ("model", "iron_smelting"),"amount": 1.0, "type": "technosphere"},
            ]
        },
        ("model", "mining"): {
            "name": "mining", 
            "type": "process", 
            "exchanges": [
                {"input": ("eco", "carbon_dioxide"),"amount": 2.0, "type": "biosphere"},
                {"input": ("model", "electricity_production"),"amount": 0.02, "type": "technosphere"},
                {"input": ("model", "iron_smelting"),"amount": 0.002, "type": "technosphere"},
            ],
            "products": [
                {"code": "coal", "name":"coal", "allocation": 1.0, "exchanges": [
                    {"input": ("model", "coal"),"amount": 5.0, "type": "production"},
                ]},
                {"code": "iron_ore", "name":"iron_ore", "allocation": 1.0, "exchanges": [
                    {"input": ("model", "iron_ore"),"amount": 5.0, "type": "production"},
                ]},
            ]
        },
        ("model", "iron_smelting"): {
            "name": "iron_smelting", 
            "type": "process", 
            "exchanges": [
                {"input": ("eco", "carbon_dioxide"),"amount": 1.0, "type": "biosphere"},
                {"input": ("model", "iron_ore"),"amount": 2, "type": "technosphere"},
            ]
        },
    },
)

Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 3/3 [00:00<?, ?it/s]


Vacuuming database 


You can also create a product using the Product proxy

In [6]:
from bw2data.backends.proxies import Product, Exchange

coal = Product(
    code = "coal",
    name = "coal",
    source_code = "mining",
    database = "model",
    allocation = 1
)

Perform an LCA containing a multifunctional process

In [7]:
act = model_db.get("iron_smelting")
lca = bc.LCA({act.key: 1}, ("CO2eq",))
lca.load_lci_data(True)
lca.lci()
lca.lcia()
lca.score

1.4110305956047413

Utility: force processing the model database

In [8]:
bd.databases.set_dirty("model")
model_db.process()

Utility: raw SQL querying

In [9]:
SQL = """
        SELECT 
            e.data,
            CASE WHEN p.matrix_id IS NULL THEN a.matrix_id ELSE p.matrix_id END AS row,
            CASE WHEN q.matrix_id IS NULL THEN b.matrix_id ELSE q.matrix_id END as col,
            e.input_database, 
            e.input_code, 
            e.output_database, 
            e.output_code,
            1 AS allocation            
        FROM exchangedataset as e
        LEFT JOIN productdataset as p ON p.code == e.input_code AND p.database == e.input_database
        LEFT JOIN productdataset as q ON q.code == e.output_code AND q.database == e.output_database
        LEFT JOIN activitydataset as a ON a.code == e.input_code AND a.database == e.input_database
        LEFT JOIN activitydataset as b ON b.code == e.output_code AND b.database == e.output_database
        WHERE 
            e.output_database = ?
            AND e.type IN ('production', 'substitution', 'generic production')
"""

import pandas as pd
import sqlite3

connection = sqlite3.connect(bd.backends.sqlite3_lci_db._filepath)

pd.read_sql_query(SQL, connection, params=["model"])

,data,row,col,input_database,input_code,output_database,output_code,allocation
0,b'\x80\x04\x95S\x00\x00\x00\x00\x00\x00\x00}\x...,5,5,model,coal,model,coal,1
1,b'\x80\x04\x95W\x00\x00\x00\x00\x00\x00\x00}\x...,6,6,model,iron_ore,model,iron_ore,1
